# Freeroll
This notebook is focused on analyzing the freeroll. The goal is to understand how efficient the buggy is in preserving its forward momentum as it progresses through the freeroll.

At the high level, we will be focusing on speed to understand small differences in freeroll time, and then on acceleration and drag to understand small differences in speed.

### Reference Track
The reference track provides a consistent set of reference points for us to align our data to. While you should care that the track is generally representative of your best line, you might prefer to keep the same reference track for an entire semester so your analysis is consistent.

In [ ]:
%matplotlib widget
import json
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from glob import glob

from plot_utils import *
from constants import *

import reference
reference_distance, reference_track = reference.load(f'{DATA_DIR}/parsed/yyyy-mm-dd_hhmmss_SonOfThunderpuppy_0.csv')
print(f'Loaded {len(reference_distance)} reference points from {reference_track.data_path}')

In [ ]:
from track import Track
# Load track data and list all the processed files
file_patterns = [
    f'{DATA_DIR}/parsed/yyyy-mm-dd_*',
]
data_files = [file for pattern in file_patterns for file in glob(pattern)]
data = []
for data_file in data_files:
    label = os.path.splitext(os.path.basename(data_file))[0]
    data.append(Track(
        label, data_file, reference_track,
    ))
    print(f'Loaded {label}')

# Plotting Against Time
Time is a relative measurement. For it to be useful, you need to know what it is relative to. With GPS data, we can align our time axis to any split point we can imagine. There are a lot of variables that factor into when a buggy reaches a specific point, and the further you get from the chosen split, the more difficult it is going to be to make meaningful comparisons between rolls.

Adjust `split_idx` to align to different split points and compare your data.

Note that different rolls will enter and exit the freeroll at different times depending on how fast the buggy was rolling.

In [ ]:
plt.figure()

split_idx = 120
variables = [SPEED, 'acceleration']

for track in data:
    df = track.freeroll
    t = df[T].to_numpy()
    t0 = track.fine_splits[split_idx]
    independent = t - t0
    
    for variable in variables:
        # Plot the variable if it exists in the raw data
        if variable in df:
            dependent = df[variable].to_numpy()
            plt.plot(independent, dependent, label=f'{track} {variable} raw')
        # Plot the variable if it has a model or derivation
        if hasattr(track, variable):
            dependent = getattr(track, variable)(t)
            plt.plot(independent, dependent, label=f'{track} {variable} smoothed')

plt.axvline(x=0, color='gray', ls='--', lw=0.8)
plt.suptitle(f'{variables} vs Time (s)')
plt.legend()
plt.show()

# Plotting with Reference Points
With a stopwatch, you are generally limited to just a handful of split times, but we have literally hundreds of refrence points we could split at. By putting the distance along the reference track on the x-axis, our data is inherently aligned, and we know that we're making a fair apples to apples comparison over the entire domain.

Note that distance along individual tracks can still vary significantly, so `reference_distance` is used as the independent variable in these plots.

Speed, Acceleration, and Drag are chosen for this plot to help derive insight into how fast the buggy was traveling during the freeroll and how efficiently it converts its gravitational potential energy into forward momentum.

In [ ]:
plt.figure()
speed = plt.subplot(3,2,1)
plt.setp(speed.get_xticklabels(), visible=False)
acceleration = plt.subplot(3,2,3, sharex=speed)
acceleration.axhline(y=0, color='gray', lw=0.8)
plt.setp(acceleration.get_xticklabels(), visible=False)
drag = plt.subplot(3,2,5, sharex=speed)
birds_eye = plt.subplot(1,2,2)
for track in data:
    line, = speed.plot(reference_distance, track.speed(track.fine_splits))
    acceleration.plot(reference_distance, track.acceleration(track.fine_splits) / G)
    drag.plot(reference_distance, track.drag(track.fine_splits))
    label = f'{track.buggy_name}\n{track.date} roll {track.roll_number}'
    birds_eye.plot(track.full_dataframe[X].to_numpy(), track.full_dataframe[Y].to_numpy(), label=label)
plt.suptitle('Variable vs. Reference Distance (m)')
speed.set_ylabel('Speed (m/s)')
acceleration.set_ylabel('Accl (Gs)')
drag.set_ylabel('Drag (Gs)')
drag.set_xlabel('Reference Distance (m)')
drag.set_ylim(0, 0.05)
add_birds_eye_view(birds_eye, reference_track.split_xs, reference_track.split_ys, reference_distance)
add_crosshairs(speed, acceleration, drag)
birds_eye.legend()
plt.tight_layout(h_pad=0, pad=1)
plt.show()

# Resampling
Your GPS data should have a reported accuracy or standard deviation depending on the solver was configured. The `track.resample()` function uses these uncertainties to generate a new track that represents what it might look like if we had collected another set of GPS data for a perfectly identical roll.

### Monte Carlo Simulation
Ideally, drag would also be reported with its own uncertainty. In lieue of developing a complicated model to include error bars on our drag estimate, we can get a reasonable approximation by running a series of Monte Carlo simulations to see how our drag estimation changes if we feed it resampled data. If the resampled estimates are all tightly clustered together, we can be confident that the drag estimation is precise and accurate in that area. If the resampled estimates are spread out, a difference we see in our original comparison of two rolls may be due to random variation in the data.

You may have noticed that the spline we use to model speed is fairly aggressive in how it smooths the data. This model is tuned to give a clear and consistent estimate of drag. Drag is calculated from the derivative of speed and altitude, so it is very sensitive to over/under fitting the data. The Monte Carlo simulation helps us understand the range of possible outcomes from our estimation, and therefore how confident we should or perhaps more importantly should NOT be in conclusions that we might draw from the data.

### Confidence Intervals
Plotting Monte Carlo simulations makes for a very pretty picture, but it doesn't help us quantify the uncertainty in our estimate. The 16th quantile of the data is approximately 1 standard deviation below the median. The 50th quantile is the median value. The 84th quantile is approximately 1 standard deviation above the median. Note that the upper and lower quantiles are asymetrical about the median.

In [ ]:
original_track = data[0]
N = 100
samples = [original_track.resample() for _ in range(N)]

In [ ]:
start = time .time()
fig, (drag, birdseye) = plt.subplots(2)
alpha = 5.0 / N 
color = None
for track in samples:
    line, = drag.plot(
        reference_distance, track.drag(track.fine_splits), 
        label=track, color=color, alpha=alpha)
    if color is None:
        color = line.get_color()
birdseye.plot(original_track.freeroll[X].to_numpy(), original_track.freeroll[Y].to_numpy())
fig.suptitle(f'Monte Carlo simulation of Drag (Gs) vs. Reference Distance (m)\n{original_track.label}')
add_birds_eye_view(birdseye, reference_track.split_xs, reference_track.split_ys, reference_distance)
add_crosshairs(drag)
drag.set_ylim(0, 0.05)
drag.set_ylabel('Drag (Gs)')
drag.set_xlabel('Reference Distance (m)')
plt.tight_layout()
plt.show()
print(f'Runtime {time.time() - start}')

In [ ]:
start = time.time()
drag = pd.DataFrame()
for idx, track in enumerate(samples):
    drag[idx] = track.drag(track.fine_splits)
quantiles = drag.transpose().quantile(q=[0.16, 0.5, 0.84]).transpose()

fig, (drag, birdseye) = plt.subplots(2)
drag.plot(reference_distance, quantiles[0.16].to_numpy(), color=color, ls='--', alpha=0.8)
drag.plot(reference_distance, quantiles[0.5].to_numpy(), color=color, lw=0.8)
drag.plot(reference_distance, quantiles[0.84].to_numpy(), color=color, ls='--', alpha=0.8)
fig.suptitle(f'{original_track.label}\n16th, 50th, and 84th Quantiles of Drag (Gs) vs. Reference Distance (m)')
add_birds_eye_view(birdseye, reference_track.split_xs, reference_track.split_ys, reference_distance)
add_crosshairs(drag)
drag.set_ylim(0, 0.05)
drag.set_ylabel('Drag (Gs)')
drag.set_xlabel('Reference Distance (m)')
plt.tight_layout()
plt.show()
print(f'Runtime {time.time() - start}')